# A final "uniform" initialisation syntax ?

## Prehistory

A long long time ago, built-in types were initialized with an equal sign ("copy initilization"), objects with parenthesis ("direct initialization").

In [ ]:
class MyInt
 {
  public : 
    MyInt( int i = 0 ) { m_value = i ; }
  private :
    int m_value ;
 } ;

In [ ]:
int i1 = 42   ; // copy initialization
MyInt i3(42)  ; // direct initialization

## C++98 generalize the use of parenthesis

Initialisation with parenthesis comes from classes. C++98 made it usable for builtin types.

In [ ]:
int i1(42)    ; // direct initialization
MyInt i2(42)  ; // direct initialization

**BUT** this was not solving some syntax analysis problems, such as the "most vexing parse".

In [ ]:
MyInt mi()  ;

## C++11 generalize the use of curly braces

#### It was already possible for aggregates and arrays

In [ ]:
struct MyPoint
 { double x, y ; } ;

In [ ]:
MyPoint p = { 4.1, 5.3 } ;
int t[] = { 3, 5, 9 } ;

#### C++11 allows it for any type, with or without `=`

In [ ]:
int i1 = { 42 }   ; // copy brace initialisation
int i2 { 42 }     ; // direct brace initialisation
MyInt i3 = { 42 } ; // copy brace initialisation
MyInt i4 { 42 }   ; // direct brace initialisation

#### Benefit: the use of empty braces initialize to 0

In [ ]:
int n {} ; // same as: int n(0) ;

#### Benefit: the use of braces forbid type narrowing

In [ ]:
int i1 = 3.14   ; // Tolerated
int i2 { 3.14 } ; // Forbidden
short s1 = 100000   ; // Tolerated
short s2 { 100000 } ; // Forbidden

## The new collection `std::initializer_list<T>` 

#### The deduced type when combining `auto` and copy brace initialization

In [ ]:
#include <iostream>

In [ ]:
auto params = { 1, 2, 3, 4, 5 } ;
for ( int param : params )
 { std::cout << param << " " ; }
std::cout << std::endl ;

For the unambiguous deduction of `T`, all the elements between the braces must be of the same type.

#### A class can have a constructor from a `std::initializer_list<T>`

In [ ]:
#include <iostream>
#include <vector>

In [ ]:
template <typename T>
class MyCollection
 {
  public:
    MyCollection( std::initializer_list<T> const & elements )
     {
      for ( auto element : elements )
       {  m_elements.push_back(element) ; }
     }
    void display()
     {
      for ( auto element : m_elements )
       { std::cout<<element<<" " ; }
      std::cout<<std::endl ;
     }
  private :
    std::vector<T> m_elements ;
 } ;

In [ ]:
MyCollection col { 1, 2, 3, 4, 5 } ;
col.display() ;

In case of brace initialization, this constructor has priority over any other.

#### All standard collections now have a constructor from a `std::initializer_list<T>`

In [ ]:
#include <iostream>
#include <vector>

In [ ]:
std::vector<int> col { 1, 2, 3, 4, 5 } ;
for ( auto element : col )
 { std::cout << element << " " ; }
std::cout << std::endl ;

#### Yet the others constructors are sometimes still useful

In [ ]:
#include <iostream>
#include <vector>

In [ ]:
std::vector<int> col1 { 2, 10 } ; // list of elements
for ( auto element : col1 )
 { std::cout << element << " " ; }
std::cout << std::endl ;

In [ ]:
std::vector<int> col2 ( 3, 5 ) ;  // size then default value
for ( auto element : col2 )
 { std::cout << element << " " ; }
std::cout << std::endl ;

## Best practice

The current recommended syntax is the **direct brace initialization**... but there are still few usages for all the other kind of initializations. "uniformity" is not really achieved.

## Exercise

In the class `BrokenLine` below, replace the `add()` and `close()` methods by a single constructor taking an `std::list_initializer<Point<Real>>` as first argument, and a boolean as second argument, the later saying if the line is closed or not. Simplify the main accordingly.

In [ ]:
%%file tmp.initialisation.cpp

#include <cassert>
#include <iostream>
#include <list>

template< typename Real >
class Point
 {
  public :
    Point( Real x, Real y )
     : m_x{x}, m_y{y} {}
    friend std::ostream & operator<<( std::ostream & os, Point const & p )
     { return (os<<"{"<<p.m_x<<","<<p.m_y<<"}") ; }
  private :
    Real m_x, m_y ;
 } ;

enum class BrokenLineKind { Empty, Opened, Closed } ;

template< typename Real >
class BrokenLine
 {
  public :
    void add( Point<Real> p )
     { m_points.push_back(p) ; m_kind = BrokenLineKind::Opened ; }
    void close()
     { assert(m_points.size()>0) ; m_kind = BrokenLineKind::Closed ; }
    friend std::ostream & operator<<( std::ostream & os, BrokenLine const & bl )
     {
      char sep { '{' } ;
      for ( auto p : bl.m_points )
       {  os<<sep<<p ; sep = ',' ; }
      if (bl.m_kind==BrokenLineKind::Closed)
       { os<<sep<<bl.m_points.front() ; }
      return (os<<'}') ;
     }
  private :
    std::list<Point<Real>> m_points ;
    BrokenLineKind m_kind = BrokenLineKind::Empty ;
 } ;
 
int main()
 {
  BrokenLine<double> bl ;
  bl.add(Point<double>{1,1}) ;
  bl.add(Point<double>{2,3}) ;
  bl.add(Point<double>{4,2}) ;
  bl.close() ;
  std::cout<<bl<<std::endl ;
 }

In [ ]:
!rm -f tmp.initialisation.exe && g++ -std=c++17 tmp.initialisation.cpp -o tmp.initialisation.exe

In [ ]:
!./tmp.initialisation.exe

© *CNRS 2020*  
*Assembled and written by David Chamont, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)